# Clase Práctica #5 (Compilación)

En esta clase estaremos adaptando el evaluador de expresiones aritméticas para trabajar sobre un _AST (Abstact Syntax Tree)_. Recordemos que el AST posee una estructura más cómoda para evaluar las reglas semánticas que el árbol de derivación. Además, evaluar en el AST en lugar que desde las reglas de la gramática atributada, simplifica significativamente la implementación de las reglas semánticas.

## Jerarquía del AST

Definamos una jerarquía de clases para los nodos del _AST_ del lenguaje de expresiones aritméticas. Utilizaremos las clases `Node` y `BinaryNode` como definiciones abstractas para agrupar y compactar la implementación de sus descendientes. Los nodos del AST serán exclusivamente instancias de `ConstantNumberNode`, `PlusNode`, `MinusNode`, `StarNode` y `DivNode`.

In [ ]:
class Node:
    def evaluate(self):
        raise NotImplementedError()

class ConstantNumberNode(Node):
    def __init__(self, lex):
        self.lex = lex
        self.value = float(lex)
        
    def evaluate(self):
        # Insert your code here!!!
        return self.value
        

class BinaryNode(Node):
    def __init__(self, left: Node, right: Node):
        self.left = left
        self.right = right
        
    def evaluate(self):
        ## Insert your code here!!!
        # lvalue = ???
        # rvalue = ???
        lvalue = self.left.evaluate()
        rvalue = self.right.evaluate()
        return self.operate(lvalue, rvalue)
    
    @staticmethod
    def operate(lvalue, rvalue):
        raise NotImplementedError()
        

class PlusNode(BinaryNode):
    @staticmethod
    def operate(lvalue, rvalue):
        # Insert your code here!!!
        return lvalue + rvalue
        

class MinusNode(BinaryNode):
    @staticmethod
    def operate(lvalue, rvalue):
        # Insert your code here!!!
        return lvalue - rvalue

class StarNode(BinaryNode):
    @staticmethod
    def operate(lvalue, rvalue):
        # Insert your code here!!!
        return lvalue * rvalue

class DivNode(BinaryNode):
    @staticmethod
    def operate(lvalue, rvalue):
        # Insert your code here!!!
        return lvalue / rvalue


: 

Veamos como luce una instancia concreta de un AST de expresiones. Nótese que la precedencia de los operadores debe seguir atrapada en el AST puesto que solo se desecharon los atributos sintácticos.

In [ ]:
from cmp.ast import get_printer
printer = get_printer(AtomicNode=ConstantNumberNode, BinaryNode=BinaryNode)

print(printer(
    PlusNode(
        MinusNode(
            ConstantNumberNode('5'),
            ConstantNumberNode('6')
        ), ConstantNumberNode('9')
    )
))

: 

## Construcción del AST

Pasemos a definir la gramática del lenguaje de expresiones aritméticas junto con las reglas semánticas para formar el _AST_. Las reglas quedarán muy similares a las de la clase anterior, pero esta vez en lugar de operar los valores, construiremos el nodo del AST que denota la operación.

In [ ]:
from cmp.pycompiler import Grammar
from cmp.utils import pprint, inspect

G = Grammar()
E = G.NonTerminal('E', True)
T, F, X, Y = G.NonTerminals('T F X Y')
plus, minus, star, div, opar, cpar, num = G.Terminals('+ - * / ( ) num')

############################ BEGIN PRODUCTIONS ############################
# ======================================================================= #
#                                                                         #
# ========================== { E --> T X } ============================== #
#                                                                         #
E %= T + X, lambda h,s: s[2], None, lambda h,s: s[1]                    #
#                                                                         #
# =================== { X --> + T X | - T X | epsilon } ================= #
#                                                                         #
X %= plus + T + X, lambda h,s: s[3], None, None, lambda h,s: PlusNode(s[2], h[0])                             
X %= minus + T + X, lambda h,s: s[3], None, None, lambda h,s: MinusNode(h[0], s[2])                              
X %= G.Epsilon, lambda h,s: h[0]                                                    
#                                                                         #
# ============================ { T --> F Y } ============================ #
#                                                                         #
T %= F + Y, lambda h,s: s[2], None, lambda h,s: s[1]                                            
#                                                                         #
# ==================== { Y --> * F Y | / F Y | epsilon } ================ #
#                                                                         #
Y %= star + F + Y, lambda h,s: s[3], None, None, lambda h,s: StarNode(h[0], s[2])                             
Y %= div + F + Y, lambda h,s: s[3], None, None, lambda h,s: DivNode(h[0], s[2])                              
Y %= G.Epsilon, lambda h,s: h[0]                                                    
#                                                                         #
# ======================= { F --> num | ( E ) } ========================= #
F %= num, lambda h,s: ConstantNumberNode(s[1]), None                                                    
F %= opar + E + cpar, lambda h,s: s[2], None, None, None                           
#                                                                         #
# ======================================================================= #
############################# END PRODUCTIONS #############################


: 

In [ ]:

# from typing import Iterator


# class TokensIterator:
#     def __init__(self, elements: Iterator[Token] ) -> None:
#         self.elements = elements
#         self.current = None

#     def next(self):
#         self.current = next(self.elements)
#         return self.current

: 

In [ ]:
# from cmp.pycompiler import Terminal, NonTerminal, AttributeProduction, EOF
# from typing import List

# def metodo_predictivo_con_evaluacion(G: Grammar, M):

#     def parser(tokens: TokensIterator):
        
#         stack: List[NonTerminal|Terminal] = [G.startSymbol]
#         cursor = 0
        
#         while True:
#             top = stack.pop()
#             token = tokens.current

#             if top.IsNonTerminal:
#                 try:
#                     print('repinga', M)
#                     print('eje', type(top), type(token.token_type))
#                     print('la tabla', top, token.token_type)
                    
#                     key = (top, token.token_type)
#                     print('cojones ', M[key])
#                     production: AttributeProduction = M[key][0]
#                 except KeyError:
#                     raise Exception("No se puede reconocer la cadena")
                
#                 yield production

#                 for symbol in reversed(production.Right):
#                     stack.append(symbol)
            
#             if len(stack)==0:
#                 break

#     return parser

: 

In [ ]:
# def lazy_evaluate_parse(G, M, parser, tokens):
#     tokens_iterator = TokensIterator(iter(tokens))
#     tokens_iterator.next()
#     left_parse = parser(tokens_iterator)

#     if not parser or not tokens:
#         return

#     final_value = None
    
#     while True:
#         try:
#             result = next(lazy_evaluate(next(left_parse), left_parse, tokens_iterator))
#             final_value = result
#         except StopIteration:
#             break
    
#     assert isinstance(tokens_iterator.current.token_type, EOF)
    
#     return final_value
    

# def lazy_evaluate(production: AttributeProduction, left_parse, tokens: TokensIterator, inherited_value=None):
#     head, body = production
#     attributes = production.attributes

#     synteticed = [None for x in attributes]
#     inherited = [None for x in attributes]
    
#     if not inherited_value is None:
#         inherited[0] = inherited_value
    
#     for i, symbol in enumerate(body, 1):
#         if symbol.IsTerminal:
#             assert inherited[i] is None
#             token = tokens.current
            
#             if token.token_type == num:
#                 synteticed[i] = float(token.lex)
#             else: 
#                 synteticed[i] = token.lex
            
#             tokens.next()
#         else:
            
#             next_production = next(left_parse)
#             # print(production)
#             # print('symbol', symbol)
#             # print('next', next_production)
#             assert symbol == next_production.Left

#             if not attributes[i]:
#                 synteticed[i] = next(lazy_evaluate(next_production, left_parse, tokens))
#             else:
#                 attr_value = attributes[i](inherited, synteticed)
#                 synteticed[i] = next(lazy_evaluate(next_production, left_parse, tokens, attr_value))
    
#     yield attributes[0](inherited, synteticed)

: 

: 

Ensamblemos el pipeline de evaluación con los elementos que hemos ido implementando a lo largo de las pasadas clases.

Se realizará la siguiente cadena de transformaciones:
```
Entrada -> Tokens -> Parse Izquierdo -> AST -> Resultado
```    

In [ ]:
from cmp.utils import Token
from cmp.languages import BasicHulk
from cmp.tools.parsing import build_parsing_table, metodo_predictivo_no_recursivo
from cmp.tools.evaluation import evaluate_parse

hulk = BasicHulk(G)
firsts = hulk.firsts
follows = hulk.follows
tokenize_text = hulk.tokenizer

M = build_parsing_table(G, firsts, follows)
parser = metodo_predictivo_no_recursivo(G, M, firsts, follows)
# custom_parser = metodo_predictivo_con_evaluacion(G, M)

def run_pipeline(text, tokenizer, value, parser, formatter):
    tokens = tokenizer(text)
    pprint(tokens, '================Tokens================')
    left_parse = parser(tokens)
    pprint(left_parse, '==============Left-Parse==============')
    ast = evaluate_parse(left_parse, tokens)
    pprint(formatter(ast), '=================AST==================')
    result = ast.evaluate()

    pprint(f'{text} = {result}', '================Result================')
    assert result == value

# def run_custom_pipeline(text, tokenizer, value, parser, formatter):
#     tokens = tokenize_text(text)
#     print('dame los putos tokens', tokens)
#     result = lazy_evaluate_parse(G, M, parser, tokens)
#     pprint(f'{text} = {result.value}', '================Result================')
#     assert result.value == value

: 

Comprobemos que la asociatividad de los operadores no se perdió.

In [ ]:
run_pipeline('1 - 1 - 1', tokenize_text, -1,  parser, printer)

: 

In [ ]:
run_pipeline('1 - ( 1 - 1 )', tokenize_text, 1,  parser, printer)

: 

## Adicionando operador _potencia_

Añadamos el operador potencia al lenguaje. Para ello, realizaremos las modificaciones pertinentes a cada una de las fases del evaluador. No será necesario **copia y pegar** código de otras clases, ni modificar el código fuente del módulo `cmp` que se distribuye junto al _notebook_.

Usaremos el símbolo `^` para denotar al operador potencia. Este es un operador binario que computa $a^b$ siendo, `a` y `b` los operandos izquierdo y derecho respectivamente. Por ejemplo, `2 ^ 4` computa $2^4$. El operador potencia asocia hacia la derecha (contrario a los operadores: +, -, \* y /). Por tanto, `4 ^ 3 ^ 2` computa $4^{3^2} = 4^9$ en lugar de $(4^3)^2 = 4^6$.

In [ ]:
class PowNode(BinaryNode):
    @staticmethod
    def operate(lvalue, rvalue):
        # Insert your code here!!!
        return lvalue ** rvalue

: 

In [ ]:
G = Grammar()
E = G.NonTerminal('E', True)
T, F, X, Y, A, Z = G.NonTerminals('T F X Y A Z')
plus, minus, star, div, opar, cpar, num, pow = G.Terminals('+ - * / ( ) num ^')

############################ BEGIN PRODUCTIONS ############################
# ======================================================================= #
#                                                                         #
# ========================== { E --> T X } ============================== #
#                                                                         #
E %= T + X, lambda h,s: s[2], None, lambda h,s: s[1]                    #
#                                                                         #
# =================== { X --> + T X | - T X | epsilon } ================= #
#                                                                         #
X %= plus + T + X, lambda h,s: s[3], None, None, lambda h,s: PlusNode(s[2], h[0])                             
X %= minus + T + X, lambda h,s: s[3], None, None, lambda h,s: MinusNode(h[0], s[2])                              
X %= G.Epsilon, lambda h,s: h[0]                                                    
#                                                                         #
# ============================ { T --> F Y } ============================ #
#                                                                         #
T %= F + Y, lambda h,s: s[2], None, lambda h,s: s[1]                                            
#                                                                         #
# ==================== { Y --> * F Y | / F Y | epsilon } ================ #
#                                                                         #
Y %= star + F + Y, lambda h,s: s[3], None, None, lambda h,s: StarNode(h[0], s[2])                             
Y %= div + F + Y, lambda h,s: s[3], None, None, lambda h,s: DivNode(h[0], s[2])                           
Y %= G.Epsilon, lambda h,s: h[0]                                                    
#                                                                         #
# ======================= { F --> num | ( E ) } ========================= #
F %= A + Z, lambda h,s: s[2], None,  lambda h,s: s[1]
A %= num, lambda h,s: ConstantNumberNode(s[1]), None                                                    
A %= opar + E + cpar, lambda h,s: s[2], None, None, None      
Z %= pow + F, lambda h,s: PowNode(h[0], s[2]), None, None
Z %= G.Epsilon, lambda h,s: h[0]                  
#                                                                         #
# ======================================================================= #
############################# END PRODUCTIONS #############################




: 

In [ ]:
from cmp.languages import UnknownToken


fixed_tokens = { lex: Token(lex, G[lex]) for lex in '+ - * / ( ) ^'.split() }
def tokenizer(G, fixed_tokens):
        fixed_tokens = fixed_tokens

        def tokenize_text(text: str):
            tokens = []
            splitted_text = text.split()
            for i, item in enumerate(splitted_text):
                try:
                    float(item)
                    token = Token(item, G['num'])
                except ValueError:
                    try:
                        token = fixed_tokens[item]
                    except KeyError:
                        if i == len(splitted_text)-1:
                            token = Token(item, G['id'])
                        elif i < len(splitted_text) - 1 and splitted_text[i + 1] in ('+', '-', '*', '/', '(', ')', '^', '='):
                            token = Token(item, G['id'])
                        else:
                            token = UnknownToken(item)
                tokens.append(token)
            eof = Token('$', G.EOF)
            tokens.append(eof)
            return tokens

        return tokenize_text


: 

Como ayuda se proveen los conjunto _First_ y _Follow_ precomputados de lo que consideramos la gramática **natural** a obtener. De igual forma se tiene la tabla y parser LL(1). Siéntase libre de utilizar el código siguiente pero puede reemplazarlo por sus propias implementaciones en caso de que no pueda (o no entienda) cómo utilizarlo.

In [ ]:
from cmp.languages import PowHulk
pow_hulk = PowHulk(G)

firsts = pow_hulk.firsts
follows = pow_hulk.follows

tokenize_text = tokenizer(G, fixed_tokens)
print('tokens', tokenize_text)

from cmp.tools.parsing import build_parsing_table, metodo_predictivo_no_recursivo
M = build_parsing_table(G, firsts, follows)
print('mm', M)
parser = metodo_predictivo_no_recursivo(G, M, firsts, follows)
# custom_parser = metodo_predictivo_con_evaluacion(G, M)
print(parser)
v = (E, num)
print(M[v])
for x in M:
    print(x == v)


: 

**Comprobemos que el operador potencia asocia hacia la derecha.**

In [ ]:
run_pipeline('4 ^ 3 ^ 2', tokenize_text, 262144, parser, printer)

: 

**Comprobemos que tiene más precedencia que el resto de los operadores.**

In [ ]:
run_pipeline('2 * 3 ^ 4 + 1 * 5', tokenize_text, 167, parser, printer)

: 

**Comprobemos que puede subordinarse a otros operadores usando paréntesis.**

In [ ]:
run_pipeline('3 ^ ( 1 + 1 ) ^ 2', tokenize_text, 81, parser, printer)

: 

## Adicionando _declaraciones de variables_

Añadamos variables al lenguaje de expresiones para acercarnos más a `HULK`. Para ello, agregue la expresión `let-in` al lenguaje. Dicha expresión sigue la siguiente sintaxis:

```
let
    <declaration-list>
in
    <expr>
```
donde `<expr>` denota cualquier expresión del lenguaje (incluyendo el propio `let-in`) y `<declaration-list>` representa una secuencia de declaraciones de la forma `<id> = <expr>` separadas por "`,`".

El valor de evaluación de la expresión `let-in` es el valor de evaluación de `<expr>`.  
Las variables declaradas en `<declaration-list>` serán visibles a partir de su declaración pero únicamente dentro de la expresión `let-in` que las contiene (incluye `<expr>`). Si `<expr>` contiene a su vez una expresión `let-in`, la declaración de una variable con el mismo nombre que una en el `let-in` padre **ocultará** la del padre.  
Por ejemplo, la expresión:
```
let
    x = 1,
    y = 2
in
    3 + (let x = 4, z = 5 in x + y + z) + x
```
equivalente a:
```
let x = 1, y = 2 in 3 + (let x = 4, z = 5 in x + y + z) + x
```
evalúa `15`.

In [ ]:
class LetHulk:
    def __init__(self, G: Grammar, fixed_tokens) -> None:
        self.G = G,
        self.fixed_tokens = fixed_tokens

class VariableNode():
    def __init__(self,id,dic):
        self.id = id 
        self.dic = dic
    @staticmethod
    def evaluate(dic , id):
        return dic[id]
class AssignNode():

    def __init__(self,id, value , assingNode):
        self.value = value
        self.id = id
        self.assingNode = assingNode
        self.dic = {}
        self.dic[id] = value
        if assingNode:
            print(type(assingNode))
            print(assingNode.dic)
            for i,j in assingNode.dic:
                print(i, 'valor')
                print(j, 'valoooooor')
                self.dic[i] = j

    def evaluate(self):
        return self.dic

: 

In [ ]:
G = Grammar()
E = G.NonTerminal('E', True)
T, F, X, Y, A, Z , V , L ,I = G.NonTerminals('T  F X Y A Z V L I')
plus, minus, star, div, opar, cpar, num, pow , equal, let, inh, id, comma = G.Terminals('+ - * / ( ) num ^ = let in id ,')

############################ BEGIN PRODUCTIONS ############################
# ======================================================================= #
#                                                                         #
# ========================== { E --> T X } ============================== #
#                                                                         #
E %= T + X, lambda h,s: s[2], None, lambda h,s: s[1]                    #
#                                                                         #
# =================== { X --> + T X | - T X | epsilon } ================= #
#                                                                         #
X %= plus + T + X, lambda h,s: s[3], None, None, lambda h,s: PlusNode(s[2], h[0])                             
X %= minus + T + X, lambda h,s: s[3], None, None, lambda h,s: MinusNode(h[0], s[2])                              
X %= G.Epsilon, lambda h,s: h[0]                                                    
#                                                                         #
# ============================ { T --> F Y } ============================ #
#                                                                         #
T %= F + Y, lambda h,s: s[2], None, lambda h,s: s[1]                                            
#                                                                         #
# ==================== { Y --> * F Y | / F Y | epsilon } ================ #
#                                                                         #
Y %= star + F + Y, lambda h,s: s[3], None, None, lambda h,s: StarNode(h[0], s[2])                             
Y %= div + F + Y, lambda h,s: s[3], None, None, lambda h,s: DivNode(h[0], s[2])                           
Y %= G.Epsilon, lambda h,s: h[0]                                                    
#                                                                         #
# ======================= { F --> AZ } ===================================#
#
F %= A + Z, lambda h,s: s[2], None,  lambda h,s: s[1]
#
#
#======================{A --> num | (E) | let VI | id}=================================#
#
#
A %= num, lambda h,s: ConstantNumberNode(s[1]), None                                                    
A %= opar + E + cpar, lambda h,s: s[2], None, None, None  
A %= let + V + I , lambda h,s: s[3], None, None, lambda h,s: s[2]
A %= id, lambda h,s: VariableNode(h[0], s[1] ), None
# 
# ===================={Z --> ^F | epsilon}================================#    
Z %= pow + F, lambda h,s: PowNode(h[0], s[2]), None, None
Z %= G.Epsilon, lambda h,s: h[0]                 
#                                                                         
#==========================={V --> id= EL}================================#
#
#
V %= id + equal + E + L , lambda h,s : AssignNode(s[1],s[3], s[4]), None, None,None, None
#
#
#==========================={L --> ,V | epsilon}================================#
L %= comma + V , lambda h,s : s[2], None, None
L %= G.Epsilon, lambda h,s : h[0]
#
#
#==========================={I --> inh E}================================#
#
I %= inh + E , lambda h,s : s[2], None, None
#
############################# END PRODUCTIONS #############################

: 

In [ ]:
# Computes First(alpha), given First(Vt) and First(Vn) 
# alpha in (Vt U Vn)*
from cmp.pycompiler import Sentence
from cmp.utils import ContainerSet


def compute_local_first(firsts, alpha: Sentence):
    first_alpha = ContainerSet()
    
    try:
        alpha_is_epsilon = alpha.IsEpsilon
    except:
        alpha_is_epsilon = False
    
    ###################################################
    # alpha == epsilon ? First(alpha) = { epsilon }
    ###################################################
    #                   <CODE_HERE>                   #
    ###################################################

    # print('FIRSTs ', firsts)
    # print('alpha, ', alpha)
    if alpha_is_epsilon:
        first_alpha.set_epsilon()

    else:
        for i, x in enumerate(alpha):
            if first_alpha.contains_epsilon:
                first_alpha.set_epsilon(False)
            if x.IsTerminal:
                first_alpha.update(ContainerSet(x))
                break
            else:
                first_alpha.hard_update(firsts[alpha[i]])
                if not first_alpha.contains_epsilon:
                    break
    
    ###################################################
    # alpha = X1 ... XN
    # First(Xi) subconjunto First(alpha)
    # epsilon pertenece a First(X1)...First(Xi) ? First(Xi+1) subconjunto de First(X) y First(alpha)
    # epsilon pertenece a First(X1)...First(XN) ? epsilon pertence a First(X) y al First(alpha)
    ###################################################
    #                   <CODE_HERE>                   #
    ###################################################
    # First(alpha)
    return first_alpha

: 

In [ ]:
# Computes First(Vt) U First(Vn) U First(alpha)
# P: X -> alpha
def compute_firsts(G):
    firsts = {}
    change = True
    
    # init First(Vt)
    for terminal in G.terminals:
        firsts[terminal] = ContainerSet(terminal)
        
    # init First(Vn)
    for nonterminal in G.nonTerminals:
        firsts[nonterminal] = ContainerSet()
    
    while change:
        change = False
        
        # P: X -> alpha
        for production in G.Productions:
            X = production.Left
            alpha = production.Right
            
            # get current First(X)
            first_X = firsts[X]
                
            # init First(alpha)
            try:
                first_alpha = firsts[alpha]
            except KeyError:
                first_alpha = firsts[alpha] = ContainerSet()
            
            # CurrentFirst(alpha)???
            local_first = compute_local_first(firsts, alpha)
            
            # update First(X) and First(alpha) from CurrentFirst(alpha)
            change |= first_alpha.hard_update(local_first)
            change |= first_X.hard_update(local_first)
                    
    # First(Vt) + First(Vt) + First(RightSides)
    return firsts

: 

In [ ]:
from itertools import islice

def compute_follows(G, firsts):
    follows = { }
    change = True
    
    local_firsts = {}
    
    # init Follow(Vn)
    for nonterminal in G.nonTerminals:
        follows[nonterminal] = ContainerSet()
    follows[G.startSymbol] = ContainerSet(G.EOF)
    
    while change:
        change = False
        
        # P: X -> alpha
        for production in G.Productions:
            X = production.Left
            alpha = production.Right
            
            follow_X = follows[X]
            
            ###################################################
            # X -> zeta Y beta
            # First(beta) - { epsilon } subset of Follow(Y)
            # beta ->* epsilon or X -> zeta Y ? Follow(X) subset of Follow(Y)
            ###################################################
            #                   <CODE_HERE>                   #
            ###################################################
            if alpha.IsEpsilon:
                continue


            for i, symbol in enumerate(alpha):
                if(symbol.IsTerminal): 
                    continue
                if(symbol.IsNonTerminal):
                    
                    if( i + 1 < len(alpha)):
                        
                        next_symbol = alpha[i+1]
                        
                            
                        local_first = firsts[next_symbol]
                           
                               
                        change |= follows[symbol].update(local_first)
                        if local_first.contains_epsilon:
                            change |= follows[symbol].update(follow_X)
                    
                    else:
                        follows[alpha[-1]] = follow_X
            ###################################################
            
    # Follow(Vn)
    return follows

: 

In [ ]:
fixed_tokens = { lex: Token(lex, G[lex]) for lex in '+ - * / ( ) ^ = let in id ,'.split() }
letHulk = LetHulk(G, fixed_tokens)
letHulkfirsts = compute_firsts(G)
letHulkFollows = compute_follows(G, letHulkfirsts)
letHulkTable = build_parsing_table(G, letHulkfirsts, letHulkFollows)
letHulkParser = metodo_predictivo_no_recursivo(G, letHulkTable, letHulkfirsts, letHulkFollows)
tokenize_text = tokenizer(G, letHulk.fixed_tokens)
        

: 

In [ ]:
expression = 'let x = 1 , y = 2 in 3 + ( let x = 4 , z = 5 in x + y + z ) + x'
run_pipeline(expression, tokenize_text, 15, letHulkParser, printer)

: 

: 